In [1]:
import pandas as pd

# Load the model outputs
gpt4_outputs = pd.read_csv("raw_results_chatgpt4_statistic.csv")
gpt4omini_outputs = pd.read_csv("raw_results_chatgpt4omini.csv")
llama3_outputs = pd.read_csv("raw_results_llama3_1.csv")
deepseek_outputs = pd.read_csv("DeepSeekV3_raw_Results.csv")

# Load the manual mapping table
mapping_table = pd.read_excel("Manual_Mapping_Table.xlsx")

# Load the ground truth
ground_truth = pd.read_excel("ground_truth.xlsx")


In [2]:
import Levenshtein

def calculate_similarity(attribute, ground_truth_attributes, threshold=0.8):
    """
    Calculate the similarity between an extracted attribute and ground truth attributes.
    Return the best match if similarity exceeds the threshold.
    """
    best_match = None
    highest_similarity = 0
    
    for gt_attribute in ground_truth_attributes:
        # Calculate normalized Levenshtein similarity
        similarity = 1 - Levenshtein.distance(attribute, gt_attribute) / max(len(attribute), len(gt_attribute))
        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = gt_attribute
    
    # Only return the match if it meets the threshold
    if highest_similarity >= threshold:
        return best_match, highest_similarity
    return None, highest_similarity


In [3]:
def evaluate_model(model_outputs, ground_truth_attributes, threshold=0.8):
    results = []
    
    for index, row in model_outputs.iterrows():
        extracted_attribute = row["Attribute"]  # Replace "Attribute" with your column name
        category = row["Category"]  # Replace "Category" with your column name
        
        # Filter ground truth attributes by category
        gt_attributes = ground_truth[ground_truth["Category"] == category]["Attribute"]
        
        # Calculate similarity
        best_match, similarity = calculate_similarity(extracted_attribute, gt_attributes, threshold)
        results.append({
            "Extracted Attribute": extracted_attribute,
            "Best Match": best_match,
            "Similarity": similarity,
            "Category": category
        })
    
    return pd.DataFrame(results)

# Evaluate each model
gpt4_similarity_results = evaluate_model(gpt4_outputs, ground_truth, threshold=0.8)
gpt4omini_similarity_results = evaluate_model(gpt4omini_outputs, ground_truth, threshold=0.8)
llama3_similarity_results = evaluate_model(llama3_outputs, ground_truth, threshold=0.8)
deepseek_similarity_results = evaluate_model(deepseek_outputs, ground_truth, threshold=0.8)

# Save results to files for analysis
gpt4_similarity_results.to_csv("gpt4_similarity_results.csv", index=False)
gpt4omini_similarity_results.to_csv("gpt4omini_similarity_results.csv", index=False)
llama3_similarity_results.to_csv("llama3_similarity_results.csv", index=False)
deepseek_similarity_results.to_csv("deepseek_similarity_results.csv", index=False)


In [5]:
def analyze_thresholds(model_outputs, ground_truth_attributes, thresholds):
    threshold_results = []
    
    for threshold in thresholds:
        # Evaluate model at current threshold
        results = evaluate_model(model_outputs, ground_truth_attributes, threshold)
        
        # Calculate metrics
        matched = results["Best Match"].notna().sum()
        total_extracted = len(results)
        total_ground_truth = len(ground_truth_attributes)
        
        precision = matched / total_extracted if total_extracted > 0 else 0
        recall = matched / total_ground_truth if total_ground_truth > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
        
        threshold_results.append({
            "Threshold": threshold,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1_score
        })
    
    return pd.DataFrame(threshold_results)

thresholds = [0.7, 0.75, 0.8, 0.85, 0.9]
gpt4_threshold_analysis = analyze_thresholds(gpt4_outputs, ground_truth, thresholds)
print(gpt4_threshold_analysis)


   Threshold  Precision    Recall  F1-Score
0       0.70   0.056812  0.809917  0.106176
1       0.75   0.052754  0.752066  0.098592
2       0.80   0.041739  0.595041  0.078007
3       0.85   0.019710  0.280992  0.036836
4       0.90   0.008696  0.123967  0.016251


In [6]:
gpt4omini_threshold_analysis = analyze_thresholds(gpt4omini_outputs, ground_truth, thresholds)
print(gpt4omini_threshold_analysis)

   Threshold  Precision    Recall  F1-Score
0       0.70       0.23  0.190083  0.208145
1       0.75       0.22  0.181818  0.199095
2       0.80       0.16  0.132231  0.144796
3       0.85       0.14  0.115702  0.126697
4       0.90       0.14  0.115702  0.126697


In [7]:
llama3_threshold_analysis = analyze_thresholds(llama3_outputs, ground_truth, thresholds)
print(llama3_threshold_analysis)

   Threshold  Precision    Recall  F1-Score
0       0.70   0.044355  0.454545  0.080823
1       0.75   0.041129  0.421488  0.074945
2       0.80   0.032258  0.330579  0.058780
3       0.85   0.014516  0.148760  0.026451
4       0.90   0.004839  0.049587  0.008817


In [8]:
deepseek_threshold_analysis = analyze_thresholds(deepseek_outputs, ground_truth, thresholds)
print(deepseek_threshold_analysis)

   Threshold  Precision    Recall  F1-Score
0       0.70   0.103093  0.826446  0.183318
1       0.75   0.100000  0.801653  0.177819
2       0.80   0.081443  0.652893  0.144821
3       0.85   0.070103  0.561983  0.124656
4       0.90   0.058763  0.471074  0.104491


In [9]:
# Extract unique attributes for each model
gpt4_attributes = list(set(gpt4_outputs["Attribute"]))
gpt4omini_attributes = list(set(gpt4omini_outputs["Attribute"]))
llama3_attributes = list(set(llama3_outputs["Attribute"]))
deepseek_attributes = list(set(deepseek_outputs["Attribute"]))

print(f"GPT-4: {len(gpt4_attributes)} attributes")
print(f"GPT-4o Mini: {len(gpt4omini_attributes)} attributes")
print(f"LLaMA 3.1: {len(llama3_attributes)} attributes")
print(f"DeepSeek-V3: {len(deepseek_attributes)} attributes")

GPT-4: 1500 attributes
GPT-4o Mini: 76 attributes
LLaMA 3.1: 1113 attributes
DeepSeek-V3: 814 attributes


In [10]:
import numpy as np
import Levenshtein

def calculate_distance_matrix(attributes):
    """
    Calculate the pairwise Levenshtein distance matrix for a list of attributes.
    """
    n = len(attributes)
    distance_matrix = np.zeros((n, n))
    
    for i, attr1 in enumerate(attributes):
        for j, attr2 in enumerate(attributes):
            if i < j:
                distance = Levenshtein.distance(attr1, attr2)
                distance_matrix[i, j] = distance
                distance_matrix[j, i] = distance
    
    # Normalize distances
    max_distance = np.max(distance_matrix)
    return distance_matrix / max_distance

# Calculate distance matrices
gpt4_distance_matrix = calculate_distance_matrix(gpt4_attributes)
gpt4omini_distance_matrix = calculate_distance_matrix(gpt4omini_attributes)
llama3_distance_matrix = calculate_distance_matrix(llama3_attributes)
deepseek_distance_matrix = calculate_distance_matrix(deepseek_attributes)

print("Pairwise distance matrices calculated.")


Pairwise distance matrices calculated.


In [11]:
from sklearn.cluster import AgglomerativeClustering

def perform_clustering(distance_matrix, attributes, n_clusters=5):
    """
    Perform Agglomerative Clustering using a precomputed distance matrix.
    """
    clustering = AgglomerativeClustering(
        n_clusters=n_clusters,
        affinity="precomputed",
        linkage="average"
    ).fit(distance_matrix)
    
    return pd.DataFrame({
        "Attribute": attributes,
        "Cluster Label": clustering.labels_
    })

# Perform clustering for each model
gpt4_clustering_results = perform_clustering(gpt4_distance_matrix, gpt4_attributes, n_clusters=5)
gpt4omini_clustering_results = perform_clustering(gpt4omini_distance_matrix, gpt4omini_attributes, n_clusters=5)
llama3_clustering_results = perform_clustering(llama3_distance_matrix, llama3_attributes, n_clusters=5)
deepseek_clustering_results = perform_clustering(deepseek_distance_matrix, deepseek_attributes, n_clusters=5)

print("Clustering completed for all models.")


Clustering completed for all models.


In [14]:
from sklearn.manifold import MDS
from sklearn.metrics import silhouette_score, davies_bouldin_score

def evaluate_clustering(distance_matrix, labels):
    """
    Evaluate clustering quality using Silhouette Score and Davies-Bouldin Index.
    """
    # Convert distance matrix to embeddings using MDS
    mds = MDS(n_components=2, dissimilarity="precomputed", random_state=42)
    embeddings = mds.fit_transform(distance_matrix)
    
    # Calculate metrics
    silhouette = silhouette_score(embeddings, labels)
    davies_bouldin = davies_bouldin_score(embeddings, labels)
    
    return silhouette, davies_bouldin

# Evaluate clustering for each model
gpt4_silhouette, gpt4_davies_bouldin = evaluate_clustering(gpt4_distance_matrix, gpt4_clustering_results["Cluster Label"])
gpt4omini_silhouette, gpt4omini_davies_bouldin = evaluate_clustering(gpt4omini_distance_matrix, gpt4omini_clustering_results["Cluster Label"])
llama3_silhouette, llama3_davies_bouldin = evaluate_clustering(llama3_distance_matrix, llama3_clustering_results["Cluster Label"])
deepseek_silhouette, deepseek_davies_bouldin = evaluate_clustering(deepseek_distance_matrix, deepseek_clustering_results["Cluster Label"])

print("GPT-4: Silhouette Score = {:.2f}, Davies-Bouldin Index = {:.2f}".format(gpt4_silhouette, gpt4_davies_bouldin))
print("GPT-4o Mini: Silhouette Score = {:.2f}, Davies-Bouldin Index = {:.2f}".format(gpt4omini_silhouette, gpt4omini_davies_bouldin))
print("LLaMA 3.1: Silhouette Score = {:.2f}, Davies-Bouldin Index = {:.2f}".format(llama3_silhouette, llama3_davies_bouldin))
print("DeepSeek-V3: Silhouette Score = {:.2f}, Davies-Bouldin Index = {:.2f}".format(deepseek_silhouette, deepseek_davies_bouldin))


GPT-4: Silhouette Score = 0.43, Davies-Bouldin Index = 2.93
GPT-4o Mini: Silhouette Score = 0.13, Davies-Bouldin Index = 0.44
LLaMA 3.1: Silhouette Score = 0.39, Davies-Bouldin Index = 1.93
DeepSeek-V3: Silhouette Score = 0.34, Davies-Bouldin Index = 1.23
